In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from pmi_case.utils import *
import pandas as pd
import plotnine as pln
import seaborn as sns
import numpy as np
import pickle as pkl
import warnings
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
from scipy import stats

## Data preparation (initial)

In [ ]:
data = pd.read_parquet('../data/dataset.parquet', engine='pyarrow')

In [ ]:
data.columns = data.columns.str.upper()
# set correct type for datetime columns
data['PRODUCTION END DATE.DAY'] = pd.to_datetime(data['PRODUCTION END DATE.DAY'], infer_datetime_format=True)
data['RELEASE DATE'] = pd.to_datetime(data['RELEASE DATE'], infer_datetime_format=True)
# BATCHID_ALT was wrogly read as a numeric column
data['BATCHID_ALT'] = data['BATCHID_ALT'].astype(np.str)
del data['BATCH SIZE UNIT']
data = data.apply(lambda x: x.str.strip() if x.dtype == 'object' else x, axis=0)

Columns that consist of one value are uninformative and are of no use in the modeling part, so we drop them in advance, not to spend unnecesarily time on them.

In [ ]:
data = drop_columns_with_single_value(data)

In [ ]:
data.dtypes.value_counts()

We check what are the types of the columns to have an info about their (most probable, since it is a heuristic) type and whether it will be used later on.

In [ ]:
column_types = data.apply(validate_column, axis=0, n_not_null=N_NOT_NULL, 
                          min_observations_in_class=MIN_OBSERVATIONS_IN_CLASS).\
    T.set_axis(['analysable', 'column_type'], axis=1)

In [ ]:
# in this case we drop all columns that have too many NAs - that many, that it is not possible to sensibly impute the missing values
data = data.loc[:, ~column_types['column_type'].str.contains('Not enough values')]
column_types = column_types.loc[~column_types['column_type'].str.contains('Not enough values')]

## EDA

Below, we plot all of the remaining columns (of appropriate types) with a graph type that is suitable to analyse the kind of data we have assumed for that variable. It will help us what can be enhanced on top of what we have previously implemented. Beware - it will result in a __long__ output.

In [ ]:
warnings.simplefilter(action='ignore')
for i, (col_name, vals) in enumerate(data.iteritems()):
    col_name_clean = col_name.replace('_', ' ').upper()
    col_type = column_types.loc[col_name]['column_type']

    if col_type in ('binary', 'multiclass', 'Not enough classes (0 or 1).'):
        p1 = (pln.ggplot(data, pln.aes(x=col_name)) +
              pln.geom_bar(fill='#ed4c9a') +
              pln.geom_text(pln.aes(y='stat(count)', label='stat(count)'), stat='count', va='bottom') +
              pln.labs(title=col_name_clean, x='', y='') +
              pln.theme_bw()
        )
        if data[col_name].unique().shape[0] > 10:
            p1 = p1 + pln.theme(axis_text_x=pln.element_text(angle=90))
    elif col_type == 'regression':
        p1 = (pln.ggplot(data, pln.aes(x=col_name)) +
              pln.geom_density(color='#ed4c9a') +
              pln.labs(title=f'Estimated density for {col_name_clean}', x=col_name_clean, y='') +
              pln.theme_bw()
              )
    else:
        continue
    display(HTML(f'<h1><center>{col_name_clean}</center></h1>'))
    print(p1)

    display(HTML('<hr>'))

Aftermath of the above round of plotting:
- drop columns where there is one, domainating value and other values occur significantly less frequently (for example: `HLAT LENGTH TARGET`);
- log `TARGET` (it has a heavy right tail);
- some variables have values that greatly deviate from the usual observations - they are likely to be erroneous entries and needs to be taken care of.

In [ ]:
# drop columns with one domainating value - it will help with smooth cross validation
data = data.loc[:, ~column_types['column_type'].str.contains('Not enough')]
# log TARGET
data['TARGET'] = np.log(data['TARGET'])

In [ ]:
# columns with erroneous entries
cols = ['PSD', 'DLBL DIAM AVG', 'DLBL DIAM LL', 'DLBL DIAM UL', 'DLBL LENGTH AVG',
        'DLBL TP PLUG BACK LENGTH AVG', 'DLBL TP BACK LENGTH STD', 'DLBL RTD AVG',
        'DLBL RTD LL', 'DLBL RTD UL', 'DLBL RTD TARGET', 'TLR DIAM AVG',
        'TLR DIAM STD', 'LL TLR DIAM', 'HLAT DIAM AVG', 'HLAT DIAM STD',
        'LL HLAT DIAM', 'UL HLAT DIAM', 'HLAT FRONT DIAM', 'LL HLAT FRONT DIAM',
        'UL HLAT FRONT DIAM', 'HLAT LENGTH', 'HLAT LENGTH STD', 'LL HLAT LENGTH',
        'UL HLAT LENGTH', 'PLLA DIAMETER', 'PLLA DIAM STD', 'PLLA LENGTH',
        'PLLA OVALITY', 'PLLA OVAL STD',
        ]

data[cols] = data[cols].apply(drop_erroneous_values, axis=0)

It also happens, that 5 objects in the sample have atypical reads for the five measurements: `LL PLLA RTD`, `UL PLLA RTD`, `PLLA RTD TARGET`, `PLLA RTD STD`, `PLLA LENGTH STD`. For these 5 objects all of the values for the given variables are zeros, whereas other objects have no-zero values. We therefore decide to drop these objects as there is probably more persistent fault with these ones.

In [ ]:
data = data.loc[~(data[['LL PLLA RTD', 'UL PLLA RTD', 'PLLA RTD TARGET', 'PLLA RTD STD', 'PLLA LENGTH STD']] == 0).all(axis=1)]

We re-draw the data once again, after our laborous corrections.

In [ ]:
warnings.simplefilter(action='ignore')
for i, (col_name, vals) in enumerate(data.iteritems()):
    col_name_clean = col_name.replace('_', ' ').upper()
    col_type = column_types.loc[col_name]['column_type']

    if col_type in ('binary', 'multiclass', 'Not enough classes (0 or 1).'):
        p1 = (pln.ggplot(data, pln.aes(x=col_name)) +
              pln.geom_bar(fill='#ed4c9a') +
              pln.geom_text(pln.aes(y='stat(count)', label='stat(count)'), stat='count', va='bottom') +
              pln.labs(title=col_name_clean, x='', y='') +
              pln.theme_bw()
              )
        if data[col_name].unique().shape[0] > 10:
            p1 = p1 + pln.theme(axis_text_x=pln.element_text(angle=90))
    elif col_type == 'regression':
        p1 = (pln.ggplot(data, pln.aes(x=col_name)) +
              pln.geom_density(color='#ed4c9a') +
              pln.labs(title=f'Estimated density for {col_name_clean}', x=col_name_clean, y='') +
              pln.theme_bw()
              )
    else:
        continue
    display(HTML(f'<h1><center>{col_name_clean}</center></h1>'))
    print(p1)

    display(HTML('<hr>'))

The clearing process that we applied above is not perfect, but it helped us to eliminate some obvious errors / wrong measurements. The data looks overall better and is more suitable for modeling.

### Missing values

Unfortunately, we cannot use `MACHINE ID` or `MATERIAL GROUP` as training features, even though they look promissing and could be impactful in the analysis. It is due to multiple infrequent values that we do not want to remove since we cannot impute these columns (infrequent groups will be problematic in cross-validation).

In [ ]:
X_fields = data.loc[:, ~data.columns.isin(
    ['BATCHID', 'BATCHID_ALT', 'GLOBAL REVISION NUMBER', 'PRODUCTION END DATE.DAY',
     'RELEASE DATE', 'BATCH STATUS', 'MACHINE ID', 'PRODUCT CODE', 'MATERIAL GROUP',
     'TARGET', 'TARGET UPPER SPECIFICATION VALUE', 'TARGET SPECIFICATION VALUE',
     'TARGET EXPONENT VALUES', 'TARGET INVERSED VALUES'])].columns
y_field = 'TARGET'

As we see below - there are columns with missing values, which would be cumbersome during the training, since the data cannot have any such entries. We will impute them using one of the moderately sophisticated methods - random-forest-based imputation. 

In [ ]:
data[X_fields].isna().sum(axis=0).plot.hist()
plt.title('Number of missing values per feature')
plt.show()

In [ ]:
from missingpy import MissForest

In [ ]:
imputer = MissForest(max_iter=15, n_estimators=200, n_jobs=4, random_state=250)
data[X_fields] = imputer.fit_transform(data[X_fields])

In [ ]:
plt.title('Number of missing values per feature')
plt.show()
data[X_fields].isna().sum(axis=0).plot.hist()  # no missing data anymore

### Outlier detection

Some of the points that we are dealing with may be multi-dimensional outliers and we wouldn't like to work with them as they are corrupting most of the algorithms that are used for classification / regression. We will get rid of them using another random-forest-based algorithm called Isolation Forest.

In [ ]:
from sklearn.ensemble import IsolationForest

In [ ]:
# we assume that ~2% of all observations are outliers - should be discussed with a subject expert
isolation_forest = IsolationForest(n_estimators=200, contamination=0.02, n_jobs=4, random_state=250)

In [ ]:
is_outlier = -1*isolation_forest.fit_predict(data[X_fields])

In [ ]:
np.unique(is_outlier, return_counts=True)

It turns out, that the algorithm considers 5 points to be outliers. We remove all of them.

In [ ]:
data = data.loc[~(is_outlier == 1)]

---

## EDA CONTINUED

Now, when all the missing values are gone and so are the outliers, we can draw correlation matrix between the numerical values and `TARGET`.

In [ ]:
plot_columns = X_fields.union(['TARGET'])

In [ ]:
sns.set_theme(style='white')
correlogram = sns.clustermap(data[plot_columns].corr(), center=0, cmap='vlag',
                             figsize=(25, 25), dendrogram_ratio=0.15,
                             row_colors=['#00c434' if col == 'TARGET' else '#c4c41b' for col in plot_columns])
correlogram.ax_row_dendrogram.remove()
plt.title('Correlogram for independent variables (golden bars on the left) and TARGET (green bar)')
plt.show()

TARGET variable seems not to be significantly correlated with many features. We can also observe clusters of tightly correlated variables. One such prominent case are `HLAT` variables. Below, we show the correlation matrix to examine further to what extent the variables are correlated and whether this can hinder the down-the-line analysis.

In [ ]:
data.filter(regex='HLAT', axis=1).corr()

It is, indeed, correlation between features of more than 0.9999 which means that these variables offer almost the same information. This entails redundancy between them. We will remove columns that have this level of co-linearity. The same situation may be present with other _categories_ of variables (like `TLR` or `PLLA`) and we will address these as well, in one go.

In [ ]:
corr_matrix = data.corr().abs()
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
to_drop = [column for column in upper.columns if any(upper[column] > 0.999)]
print(f'{len(to_drop)} columns will be removed due to excessively high correlation: {to_drop}')
data = data.drop(columns=to_drop)
X_fields = X_fields.difference(to_drop)

Since the preprocessing of the data is finalized, we store the results for further use.

In [ ]:
data_to_store = {'data': data, 'X_fields': X_fields, 'y_field': y_field}

In [ ]:
pkl.dump(data_to_store, open('../data/processed_data/data_after_preprocessing.pkl', 'wb'))

---

In [ ]:
# if needed - we can read the preprocessed data from the file
stored_data = pkl.load(open('../data/processed_data/data_after_preprocessing.pkl', 'rb'))
data, X_fields, y_field = stored_data['data'], stored_data['X_fields'], stored_data['y_field']

---

Let us also plot whether target is dependent on the weekday of the production end / release day.

In [ ]:
data['PRODUCTION_END_DATE_WEEKDAY'] = data['PRODUCTION END DATE.DAY'].dt.weekday.replace(WEEKDAY_TRANSLATION_DAY)
data['RELEASE_DATE_WEEKDAY'] = data['RELEASE DATE'].dt.weekday.replace(WEEKDAY_TRANSLATION_DAY)
weekdays = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
data['PRODUCTION_END_DATE_WEEKDAY'] = pd.Categorical(data['PRODUCTION_END_DATE_WEEKDAY'], 
                                                     categories=weekdays, ordered=True).remove_unused_categories()
data['RELEASE_DATE_WEEKDAY'] = pd.Categorical(data['RELEASE_DATE_WEEKDAY'], 
                                              categories=weekdays, ordered=True).remove_unused_categories()

In [ ]:
# ANOVA for weekday of production's end
_, p_val = stats.f_oneway(*[vals.values for _, vals in data.groupby('PRODUCTION_END_DATE_WEEKDAY')['TARGET']])

(pln.ggplot(data, pln.aes(x='PRODUCTION_END_DATE_WEEKDAY', y='TARGET')) +
 pln.geom_boxplot() +
 pln.labs(title="TARGET with respect to the weekday of the production's end\n"
                f"One-way ANOVA p-value: {np.round(p_val, 3)}", x='') +
 pln.theme_bw() +
 pln.theme(figure_size=(8, 6))
 )

In [ ]:
# ANOVA for weekday of release date
_, p_val = stats.f_oneway(*[vals.values for _, vals in data.dropna().groupby('RELEASE_DATE_WEEKDAY')['TARGET']])

(pln.ggplot(data, pln.aes(x='RELEASE_DATE_WEEKDAY', y='TARGET')) +
 pln.geom_boxplot() +
 pln.labs(title="TARGET with respect to the weekday of the release day\n"
                f"One-way ANOVA p-value: {np.round(p_val, 3)}", x='') +
 pln.theme_bw() +
 pln.theme(figure_size=(8, 6))
 )

By eyeballing, there doesn't seem to be any significant difference with respect to weekday of both: production's end and release date. This is further confirmed by one-way ANOVA statistical test which, in both cases, we do not reject the null hypothesis.

---

We can also analyse production's end's and release date's month so that we have wider, temporal, perspective on the data.

In [ ]:
data['PRODUCTION END MONTH'] = pd.Categorical(data['PRODUCTION END DATE.DAY'].dt.month, ordered=True)
data['RELEASE DATE MONTH'] = pd.Categorical(data['RELEASE DATE'].dt.month, ordered=True)

In [ ]:
# ANOVA for weekday of release date
_, p_val = stats.f_oneway(*[vals.values for _, vals in data.dropna().groupby('PRODUCTION END MONTH')['TARGET']])

(pln.ggplot(data, pln.aes(x='PRODUCTION END MONTH', y='TARGET')) +
 pln.geom_boxplot() +
 pln.labs(title="TARGET with respect to the month of the production's end\n"
                f"One-way ANOVA p-value: {np.round(p_val, 3)}", x='') +
 pln.theme_bw() +
 pln.theme(figure_size=(8, 6))
 )

In [ ]:
# ANOVA for weekday of release date
_, p_val = stats.f_oneway(*[vals.values for _, vals in data.dropna().groupby('RELEASE DATE MONTH')['TARGET']])

(pln.ggplot(data, pln.aes(x='RELEASE DATE MONTH', y='TARGET')) +
 pln.geom_boxplot() +
 pln.labs(title="TARGET with respect to the month of the release\n"
                f"One-way ANOVA p-value: {np.round(p_val, 3)}", x='') +
 pln.theme_bw() +
 pln.theme(figure_size=(8, 6))
 )

Also in these cases, there is no indication on significance of the temporal features, so we disregard them in the modeling part.

## Modeling `TARGET`

Actually - we are predicting logarithm of `TARGET`, since the values were transformed previously.

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import GridSearchCV, RepeatedKFold, KFold, train_test_split, cross_validate
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score
random_state = np.random.RandomState(1847)

In [ ]:
inner_cv = KFold(5)
k_fold = RepeatedKFold(n_splits=5, n_repeats=5)

lr_model = LinearRegression(n_jobs=2)
lr_model = make_pipeline(StandardScaler(), lr_model)  # we standardize values so that coefficients of LR are directly comparable

rf_model = RandomForestRegressor(n_estimators=201, n_jobs=2, random_state=random_state)
rf_grid = GridSearchCV(rf_model, scoring='r2', param_grid={'max_depth': [3, 5, 7]}, n_jobs=3, cv=inner_cv, refit=True)

In [ ]:
# test_size=28 so that we are left with 200 observations in the training set
train_set, holdout_set = train_test_split(data, test_size=28, random_state=random_state)

### Let's run cross-validation to initially verify models' performance

In [ ]:
train_set[X_fields].shape

In [ ]:
lr_cv_res = cross_validate(lr_model, X=train_set[X_fields].values, y=train_set[y_field].values, scoring='r2', cv=k_fold)

In [ ]:
print(f"R^2 for linear regression in 5x repeated 5-fold CV: {np.round(lr_cv_res['test_score'].mean(), 3)}"
      f" +/- {np.round(lr_cv_res['test_score'].std(), 3)}.")

Something is terribly wrong with logistic regression - we cannot trust this model in our predictions. Finding out the reason would require some drilling down.

In [ ]:
rf_cv_res = cross_validate(rf_grid, X=train_set[X_fields].values, y=train_set[y_field].values, scoring='r2', cv=k_fold)

In [ ]:
print(f"R^2 for random forest in 5x repeated 5-fold CV: {np.round(rf_cv_res['test_score'].mean(), 3)}"
      f"+/- {np.round(rf_cv_res['test_score'].std(), 3)}.")

### Train the random forest model on the whole training data and test on holdout set

In [ ]:
rf_model.fit(train_set[X_fields], train_set[y_field])

In [ ]:
print(f"R^2 on the holdout set for random forest training on the whole training set: "
      f"{np.round(r2_score(holdout_set[y_field], rf_model.predict(holdout_set[X_fields])), 3)}.")

In [ ]:
(pln.ggplot(data=pd.DataFrame({'prediction': rf_model.predict(holdout_set[X_fields]),
                               'truth': holdout_set[y_field]}), mapping=pln.aes(x='truth', y='prediction')) +
 pln.geom_point(size=3, fill='blue') +
 pln.labs(title='Predictions from random forest on holdout set vs. true values', x='Truth', y='Prediction') +
 pln.theme_bw()
 )

The predictions from the random forest are much more accurate for the holdout set than in cross-validation, which is worrying and should be closlier investigated. Such a discrepancy may be attributed to a lucky split into training and holdout set, so it is inevitable to implement a more thorough validation procedure. 

## Feature importance from random forest

In [ ]:
fi_df = pd.DataFrame({'importance': rf_model.feature_importances_,
                      'feature': train_set[X_fields].columns})

(pln.ggplot(pln.aes(x='feature', y='importance'), data=fi_df.sort_values('importance', ascending=False).iloc[:10]) +
 pln.geom_segment(pln.aes(x='feature', xend='feature', y=0, yend='importance'), color="skyblue") +
 pln.geom_point(color="blue", size=4, alpha=0.8) +
 pln.labs(x='Feature', y='Importance [Gini]', 
          title='Importance of the features for the random forest model\n (trained on 200 observations)') +
 pln.coord_flip() +
 pln.theme_bw() +
 pln.theme(
    panel_grid_major_y=pln.element_blank(),
    panel_border=pln.element_blank(),
    axis_ticks_major_y=pln.element_blank()
 )
 )

---

## Feature importance using SHAP

In [ ]:
import shap

In [ ]:
explainer = shap.TreeExplainer(rf_model)
shap_values = explainer.shap_values(holdout_set[X_fields])

In [ ]:
# explanations for the 2nd observation in the holdout set which has a (relatively) big prediction
shap.force_plot(explainer.expected_value, shap_values[2, :], np.round(holdout_set[X_fields].iloc[2, :], 3), matplotlib=True, text_rotation=20)

In [ ]:
# explanations for the last observation in the holdout set which has a strongly negative prediction
shap.force_plot(explainer.expected_value, shap_values[-1, :], np.round(holdout_set[X_fields].iloc[-1, :], 3), matplotlib=True, text_rotation=20)

In [ ]:
# explantaions for the whole holdout set
shap.force_plot(explainer.expected_value, shap_values, holdout_set[X_fields])  # doesn't work in jupyterlab :(